Righnow candidate is : SVR with PCA n_components = 10 with results 0.23706564506631833

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test.head(10)

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
5,8,y,aa,ai,e,d,x,g,s,0,...,1,0,0,0,0,0,0,0,0,0
6,10,x,b,ae,d,d,x,d,y,0,...,0,0,0,0,0,1,0,0,0,0
7,11,f,s,ae,c,d,h,d,a,0,...,0,0,1,0,0,0,0,0,0,0
8,12,ap,l,s,c,d,h,j,n,0,...,0,0,0,0,0,0,0,0,0,0
9,14,o,v,as,f,d,g,f,v,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel

In [22]:
train_labels = train.iloc[:,1]
train_feats = train.iloc[:, 2:]

test_labels = test.iloc[:,1]
test_feats = test.iloc[:, 1:]

In [23]:
train_feats_encode = []
train_feats_encode.append(train.iloc[:, 0])
for i in range(train_feats.shape[1]):
    arr = train_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    train_feats_encode.append(arr)
train_feats_encode = np.array(train_feats_encode).T

test_feats_encode = []
test_feats_encode.append(test.iloc[:, 0])
for i in range(test_feats.shape[1]):
    arr = test_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    test_feats_encode.append(arr)
test_feats_encode = np.array(test_feats_encode).T

In [24]:
train_X, test_X, train_y, test_y = train_test_split(train_feats_encode, train_labels)


*** Keras Deep Learning ***

In [25]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
act_func = 'tanh'
input_dims = train_feats_encode.shape[1]

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [26]:
# base model architecture definition
def model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [27]:
estimator = KerasRegressor(
    build_fn=model, 
    nb_epoch=100, 
    batch_size=20,
    verbose=1
)

In [28]:
estimator.fit(
    train_X, 
    train_y, 
    epochs=300,
    verbose=2,
    shuffle=True
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 377)               142506    
_________________________________________________________________
batch_normalization_4 (Batch (None, 377)               1508      
_________________________________________________________________
activation_4 (Activation)    (None, 377)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 377)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 377)               142506    
_________________________________________________________________
batch_normalization_5 (Batch (None, 377)               1508      
_________________________________________________________________
activation_5 (Activation)    (None, 377)               0         
__________

1s - loss: 85.4838 - r2_keras: 0.4912
Epoch 115/300
1s - loss: 84.6808 - r2_keras: 0.4995
Epoch 116/300
1s - loss: 84.1591 - r2_keras: 0.4995
Epoch 117/300
1s - loss: 82.7016 - r2_keras: 0.5113
Epoch 118/300
1s - loss: 82.4867 - r2_keras: 0.5187
Epoch 119/300
1s - loss: 82.2141 - r2_keras: 0.5186
Epoch 120/300
1s - loss: 82.5797 - r2_keras: 0.5189
Epoch 121/300
1s - loss: 82.3397 - r2_keras: 0.5172
Epoch 122/300
1s - loss: 81.7099 - r2_keras: 0.5155
Epoch 123/300
1s - loss: 81.9527 - r2_keras: 0.5327
Epoch 124/300
1s - loss: 80.2245 - r2_keras: 0.5423
Epoch 125/300
1s - loss: 80.2778 - r2_keras: 0.5330
Epoch 126/300
1s - loss: 80.0104 - r2_keras: 0.5316
Epoch 127/300
1s - loss: 81.5935 - r2_keras: 0.5228
Epoch 128/300
1s - loss: 81.2926 - r2_keras: 0.5255
Epoch 129/300
1s - loss: 79.8263 - r2_keras: 0.5368
Epoch 130/300
2s - loss: 80.3140 - r2_keras: 0.5201
Epoch 131/300
1s - loss: 82.2343 - r2_keras: 0.5208
Epoch 132/300
2s - loss: 82.4821 - r2_keras: 0.5142
Epoch 133/300
1s - loss: 8

1s - loss: 77.4140 - r2_keras: 0.5474
Epoch 273/300
1s - loss: 77.6039 - r2_keras: 0.5566
Epoch 274/300
1s - loss: 77.1630 - r2_keras: 0.5627
Epoch 275/300
1s - loss: 78.5327 - r2_keras: 0.5436
Epoch 276/300
1s - loss: 75.9841 - r2_keras: 0.5620
Epoch 277/300
1s - loss: 77.5444 - r2_keras: 0.5565
Epoch 278/300
1s - loss: 77.5508 - r2_keras: 0.5584
Epoch 279/300
1s - loss: 79.2670 - r2_keras: 0.5444
Epoch 280/300
1s - loss: 76.7466 - r2_keras: 0.5530
Epoch 281/300
1s - loss: 76.8680 - r2_keras: 0.5557
Epoch 282/300
1s - loss: 76.5663 - r2_keras: 0.5538
Epoch 283/300
1s - loss: 77.0937 - r2_keras: 0.5558
Epoch 284/300
1s - loss: 76.5920 - r2_keras: 0.5641
Epoch 285/300
1s - loss: 77.1521 - r2_keras: 0.5502
Epoch 286/300
1s - loss: 77.6416 - r2_keras: 0.5495
Epoch 287/300
1s - loss: 77.6912 - r2_keras: 0.5529
Epoch 288/300
1s - loss: 78.5774 - r2_keras: 0.5267
Epoch 289/300
1s - loss: 77.7936 - r2_keras: 0.5489
Epoch 290/300
1s - loss: 78.1681 - r2_keras: 0.5401
Epoch 291/300
1s - loss: 7

In [33]:
prediction_dl  = estimator.predict(train_X)
test_dl = estimator.predict(test_X)
score = r2_score(test_y, test_dl)
print("\ntrain : ", r2_score(train_y, prediction_dl)) 
print("\ntest : ", score) 

1000/1053 [===========================>..] - ETA: 0s
train :  0.106197717351

test :  0.192766033376


In [30]:
estimator.fit(
    train_X, 
    train_y, 
    epochs=300,
    verbose=1,
    shuffle=True
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 377)               142506    
_________________________________________________________________
batch_normalization_7 (Batch (None, 377)               1508      
_________________________________________________________________
activation_7 (Activation)    (None, 377)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 377)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 377)               142506    
_________________________________________________________________
batch_normalization_8 (Batch (None, 377)               1508      
_________________________________________________________________
activation_8 (Activation)    (None, 377)               0         
__________

3156/3156 [==============================] - 1s - loss: 80.1664 - r2_keras: 0.5301     
Epoch 140/300
3156/3156 [==============================] - 1s - loss: 82.6574 - r2_keras: 0.5075     
Epoch 141/300
3156/3156 [==============================] - 1s - loss: 80.8078 - r2_keras: 0.5280     
Epoch 142/300
3156/3156 [==============================] - 1s - loss: 81.7960 - r2_keras: 0.5190     
Epoch 143/300
3156/3156 [==============================] - 1s - loss: 78.5407 - r2_keras: 0.5422     
Epoch 144/300
3156/3156 [==============================] - 1s - loss: 80.1968 - r2_keras: 0.5264     
Epoch 145/300
3156/3156 [==============================] - 1s - loss: 79.9008 - r2_keras: 0.5376     
Epoch 146/300
3156/3156 [==============================] - 1s - loss: 79.5742 - r2_keras: 0.5458     
Epoch 147/300
3156/3156 [==============================] - 1s - loss: 80.5968 - r2_keras: 0.5209     
Epoch 148/300
3156/3156 [==============================] - 1s - loss: 81.5218 - r2_keras: 0.5172

3156/3156 [==============================] - 2s - loss: 79.1060 - r2_keras: 0.5456     
Epoch 220/300
3156/3156 [==============================] - 2s - loss: 78.7433 - r2_keras: 0.5462     
Epoch 221/300
3156/3156 [==============================] - 2s - loss: 80.0882 - r2_keras: 0.5326     
Epoch 222/300
3156/3156 [==============================] - 1s - loss: 80.1162 - r2_keras: 0.5279     
Epoch 223/300
3156/3156 [==============================] - 1s - loss: 81.2524 - r2_keras: 0.5239     
Epoch 224/300
3156/3156 [==============================] - 1s - loss: 78.2357 - r2_keras: 0.5536     
Epoch 225/300
3156/3156 [==============================] - 1s - loss: 78.4903 - r2_keras: 0.5466     
Epoch 226/300
3156/3156 [==============================] - 1s - loss: 78.4456 - r2_keras: 0.5301     
Epoch 227/300
3156/3156 [==============================] - 2s - loss: 78.0842 - r2_keras: 0.5451     
Epoch 228/300
3156/3156 [==============================] - 2s - loss: 78.9772 - r2_keras: 0.5394

In [31]:
test_predictions = estimator.predict(test_feats_encode)

3940/4209 [===========================>..] - ETA: 0s

In [34]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32), 'y': test_predictions})
output.to_csv('results/ensemble[%s][%s].csv' % (str(model.__class__.__name__),score), index=False)
